# Аналіз дадзеных тэгу name у ОСМ Беларусі

## Зьмест

- Праблематыка
- Спампуем дамп ОСМ
- Усталёўваем залежанасьцьі
- Пошук сувязей дзеля падтрыманьня спасылачнай цэласнасьці

## Праблематыка

У беларускім ОСМ шырока выкарыстоўваюцца беларуская і расейская мова, для іх ёсьць адпаведнікі `name:be` і `name:ru`, таксама мовы выкарыстоўваюцца ў агульных тэгах як `name`, `addr:*` і іншых. Праблематка выкарыстоўваньня аднае, ці іншае, ці абедзьвух моваў апісанае тут https://wiki.openstreetmap.org/wiki/BE:Belarus_language_issues. Незалежна ад варыянту выкарыстоўваньня мовы павінны вытрымлівацца наступныя правілы: пошук на любое мове мусіць працаваць, павінна быць магчымасьць паказываць подпісы на любой мове (ці ў арыгінале, але гэтае правіла зараз не выконваецца), павінна захоўвацца спасылкавая цэласнасьць (што можа ўплываць на папярэднія два пункты).

Гэты аналіз ставіць мэтаю знайсьці адпаведныя катэгорыі і тэгі якія ўтрымліваюць кірылічныя значэньні тэгу name і падлічыць запаўняльнасьць тэгаў name:be, name:ru.

## Спампуем дамп ОСМ

In [1]:
# !rm belarus-latest.osm.pbf
!wget --backups=1 -N https://download.geofabrik.de/europe/belarus-latest.osm.pbf
!cp belarus-latest.osm.pbf belarus-updated.osm.pbf
#!osmium fileinfo -e belarus-latest.osm.pbf
info = !osmium fileinfo -e belarus-latest.osm.pbf

--2022-06-19 11:39:53--  https://download.geofabrik.de/europe/belarus-latest.osm.pbf
Resolving download.geofabrik.de (download.geofabrik.de)... 116.202.112.212, 95.216.28.113
Connecting to download.geofabrik.de (download.geofabrik.de)|116.202.112.212|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘belarus-latest.osm.pbf’ not modified on server. Omitting download.



In [2]:
largest_node_id = [int(l.split(':')[-1].strip()) for l in info if 'Largest node ID' in l][0]
largest_way_id = [int(l.split(':')[-1].strip()) for l in info if 'Largest way ID' in l][0]
largest_rel_id = [int(l.split(':')[-1].strip()) for l in info if 'Largest relation ID' in l][0]

try:
    from belarus_utils import OverpassApiSearchEnigne
    overpass_api = OverpassApiSearchEnigne(cache=True)
    osc = overpass_api.get_updates_osc(largest_node_id, largest_way_id, largest_rel_id)
    with open('belarus-latest.osc', 'w') as h:
        h.write(osc)
except Exception as err:
#     print(err)
    raise err

In [3]:
!osmium apply-changes --overwrite -o belarus-updated.osm.pbf belarus-latest.osm.pbf belarus-latest.osc

[======================================================================] 100% =================================================>                    ]  70% 


## Загрузім дамп у postgis
- патрэбна толькі калі хочам атрымаць больш дакладныя дадзеныя, але можа не ўтрымліваць некаторыя дачыненьні

In [4]:
!PGPASSWORD=$POSTGRES_PASSWORD psql -h $POSTGRES_HOST -p $POSTGRES_POST -U $POSTGRES_USER -d $POSTGRES_DB  -c "CREATE EXTENSION IF NOT EXISTS hstore"
!PGPASSWORD=$POSTGRES_PASSWORD psql -h $POSTGRES_HOST -p $POSTGRES_POST -U $POSTGRES_USER -d $POSTGRES_DB  -c "CREATE EXTENSION IF NOT EXISTS postgis"
!PGPASSWORD=$POSTGRES_PASSWORD psql -h $POSTGRES_HOST -p $POSTGRES_POST -U $POSTGRES_USER -d $POSTGRES_DB  -c "DROP MATERIALIZED VIEW IF EXISTS planet_osm_region CASCADE"

NOTICE:  extension "hstore" already exists, skipping
CREATE EXTENSION
NOTICE:  extension "postgis" already exists, skipping
CREATE EXTENSION
NOTICE:  drop cascades to materialized view planet_osm_named_data
DROP MATERIALIZED VIEW


In [5]:
!PGPASSWORD=$POSTGRES_PASSWORD osm2pgsql -H $POSTGRES_HOST -P $POSTGRES_POST -U $POSTGRES_USER -d $POSTGRES_DB -v -l -j -G -x --hstore-add-index -C $OSM2PGSQL_CACHE -S /usr/share/osm2pgsql/default.style belarus-updated.osm.pbf

2022-06-19 11:40:23  osm2pgsql version 1.6.0
2022-06-19 11:40:23  [0] Database version: 14.3
2022-06-19 11:40:23  [0] PostGIS version: 3.2
2022-06-19 11:40:23  [0] Reading file: belarus-updated.osm.pbf
2022-06-19 11:40:23  [0] Started pool with 4 threads.
2022-06-19 11:40:23  [0] Using projection SRS 4326 (Latlong)
2022-06-19 11:40:23  [0] Using built-in tag transformations
2022-06-19 11:40:23  [0] Middle 'ram' options:
2022-06-19 11:40:23  [0]   locations: true
2022-06-19 11:40:23  [0]   way_nodes: true
2022-06-19 11:40:23  [0]   nodes: false
2022-06-19 11:40:23  [0]   untagged_nodes: true
2022-06-19 11:40:23  [0]   ways: false
2022-06-19 11:40:23  [0]   relations: false
2022-06-19 11:40:23  [0] Setting up table 'planet_osm_point'
2022-06-19 11:40:23  [0] Setting up table 'planet_osm_line'
2022-06-19 11:40:23  [0] Setting up table 'planet_osm_polygon'
2022-06-19 11:40:23  [0] Setting up table 'planet_osm_roads'
2022-06-19 11:41:50  [0] Reading input files done in 87s (1m 27s).        

## Усталюем залежнасьці

In [ ]:
!pip install pandas matplotlib psycopg2-binary https://github.com/lechup/imposm-parser/archive/python3.zip

## Вызначым катэгорыі

In [6]:
import os
import re
from collections import defaultdict, Counter

from imposm.parser import OSMParser
import psycopg2

import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', '{:.3f}'.format)


cirylic_chars = frozenset('абвгдеёжзіийклмнопрстуўфхцчшщьыъэюяАБВГДЕЁЖЗІИІЙКЛМНОПРСТУФХЦЧШЩЬЫЪЭЮЯ')


In [7]:
categories_rules = {
    'admin': [
        ['admin_level', '2'],
        ['admin_level', '4'],
        ['admin_level', '6'],
        ['admin_level', '8'],
        ['admin_level', '9'],
    ],
    'place': [
        ['place', 'city'],
        ['place', 'town'],
        ['place', 'village'],
        ['place', 'hamlet'],
        ['place', 'isolated_dwelling'],
        ['admin_level', None],
        ['boundary', 'administrative'],
        ['traffic_sign', 'city_limit'],
    ],
    'allotments': [
        ['place', 'allotments'],
        ['landuse', 'allotments'],
    ],
    'locality': [
        ['place', 'locality'],
        ['abandoned:place', None],
    ],
    'suburb': [
        ['landuse', 'commercial'],
        ['landuse', 'construction'],
        ['landuse', 'industrial'],
        ['landuse', 'residential'],
        ['landuse', 'retail'],
        ['place', None],
        ['residential', None],
        ['industrial', None],
    ],
    'highway': [
        ['highway', 'motorway'],
        ['highway', 'trunk'],
        ['highway', 'primary'],
        ['highway', 'secondary'],
        ['highway', 'tertiary'],
        ['highway', 'unclassified'],
        ['highway', 'residential'],
        ['highway', 'service'],
        ['highway', 'track'],
        ['highway', None],
        ['type', 'associatedStreet'],
        ['type', 'street'],
    ],
    'public_transport': [
        ['highway', 'bus_stop'],
        ['public_transport', None],
        ['route', None],
        ['type', 'route'],
        ['railway', None],
        ['type', 'route_master'],
        ['route_master', None],
    ],
    'infrastructure': [
        ['tunnel', None],
        ['barrier', None],
        ['power', None],
        ['bridge', None],
        ['substation', None],
        ['emergency', None],
        ['ele', None],
        ['man_made', None],
        ['embankment', None],
    ],
    'religion': [
        ['religion', None],
        ['amenity', 'place_of_worship'],
        ['amenity', 'monastery'],
        ['building', 'church'],
        ['building', 'cathedral'],
        ['building', 'chapel'],
    ],
    'education': [
        ['landuse', 'education'],
        ['amenity', 'university'],
        ['amenity', 'college'],
        ['amenity', 'school'],
        ['amenity', 'kindergarten'],
        ['building', 'university'],
        ['building', 'college'],
        ['building', 'school'],
        ['building', 'kindergarten'],
    ],
    'healthcare': [
        ['healthcare', None],
        ['amenity', 'hospital'],
        ['amenity', 'pharmacy'],
        ['amenity', 'clinic'],
        ['amenity', 'doctors'],
        ['amenity', 'dentist'],
        ['building', 'hospital'],
        ['building', 'clinic'],
    ],
    'government': [
        ['amenity', 'post_office'],
        ['amenity', 'police'],
        ['amenity', 'library'],
        ['office', 'government'],
        ['government', None],
        ['landuse', 'military'],
        ['military', None],
    ],
    'office': [
        ['office', None],
    ],
    'tourism': [
        ['tourism', None],
        ['historic', None],
        ['memorial', None],
        ['ruins', None],
        ['information', None],
        ['attraction', None],
        ['resort', None],
        ['artwork_type', None],
    ],
    'amenity': [
        ['amenity', 'cafe'],
        ['amenity', 'atm'],
        ['amenity', 'bank'],
        ['amenity', 'fast_food'],
        ['amenity', 'fuel'],
        ['amenity', 'community_centre'],
        ['amenity', 'restaurant'],
        ['amenity', 'bar'],
        ['amenity', None],
        ['shop', 'convenience'],
        ['shop', 'clothes'],
        ['shop', 'car_repair'],
        ['shop', 'hairdresser'],
        ['shop', 'chemist'],
        ['shop', 'supermarket'],
        ['shop', 'car_parts'],
        ['shop', 'furniture'],
        ['shop', 'hardware'],
        ['shop', 'kiosk'],
        ['shop', 'doityourself'],
        ['shop', 'pet'],
        ['shop', 'florist'],
        ['shop', 'beauty'],
        ['shop', 'mobile_phone'],
        ['shop', 'shoes'],
        ['shop', 'newsagent'],
        ['shop', 'electronics'],
        ['shop', 'alcohol'],
        ['shop', 'jewelry'],
        ['shop', 'mall'],
        ['shop', 'butcher'],
        ['shop', 'cosmetics'],
        ['shop', None],
        ['leisure', None],
        ['sport', None],
        ['craft', 'shoemaker'],
        ['clothes', None],
    ],
    'building': [
        ['building', 'industrial'],
        ['building', 'service'],
        ['building', 'retail'],
        ['building', 'commercial'],
        ['building', 'warehouse'],
        ['building', 'public'],
        ['building', 'dormitory'],
        ['building', 'warehouse'],
        ['building', None],
    ],
    'water': [
        ['waterway', 'drain'],
        ['waterway', 'ditch'],
        ['waterway', 'stream'],
        ['waterway', 'river'],
        ['waterway', 'canal'],
        ['waterway', None],
        ['type', 'waterway'],
        ['water', None],
        ['natural', 'water'],
        ['natural', 'spring'],
    ],
    'natural': [
        ['boundary', None],
        ['natural', None],
        ['place', 'island'],
        ['place', 'islet'],
        ['landuse', None],
    ],
}

dependants = [
    'addr:region',
    'addr:district',
    'addr:subdistrict',
    'addr:city',
    'addr:place',
    'addr:street',
    'addr2:street',
    
    'from',
    'to',
    'via',
    'destination',
    'destination:backward',
    'destination:forward',
    
    'water_tank:city',
]

usage = defaultdict(set)
categories_rules2 = {}
for category, group in categories_rules.items():
    if category not in categories_rules2:
        categories_rules2[category] = []
    for tag, value in group:
        if value is not None:
            categories_rules2[category].append([tag, True, {value}])
            usage[tag].add(value)                          
for category, group in categories_rules.items():
    if category not in categories_rules2:
        categories_rules2[category] = []
    for tag, value in group:
        if value is None:
            categories_rules2[category].append([tag, False, usage[tag]])

## Падлічам статыстыку для дампу
- дамп падліча ўсе дадзеныя, але можа быць трошкі недакладным таму што не ўлічвае грубую абрэзку Беларусі

In [8]:
key_counter = defaultdict(lambda: defaultdict(list))

categories_tags = {}
categories_rules_tags_set = {}
for category, group in categories_rules2.items():
    for tag, eq, values in group:
        if tag not in categories_tags:
            categories_tags[tag] = {category}
        else:
            categories_tags[tag].add(category)

          
def process(params):
    for _, tags, _ in params:
        if 'name' not in tags:
            continue
        if not (frozenset(tags['name']) & cirylic_chars):
            continue
        categories = {category for tag in categories_tags.keys() & tags.keys() for category in categories_tags[tag]}
        for tag in ['name', 'name:be', 'name:ru']:
            if tag not in tags:
                continue
            value = tags[tag]
            cyr = frozenset(value) & cirylic_chars
            if not cyr:
                continue
            match = False
            for category in categories:
                group = categories_rules2[category]
                category_match = False
                for i, (k, eq, vv) in enumerate(group):
                    if k not in tags:
                        continue
                    if eq:
                        if tags[k] in vv:
                            if not category_match:
                                key_counter[(category,)][tag].append(value)
                                match = category_match = True
                            key_counter[(category, i)][tag].append(value)   
                    else:
                        if tags[k] not in vv:
                            if not category_match:
                                key_counter[(category,)][tag].append(value)
                                match = category_match = True
                            key_counter[(category, i)][tag].append(value)   
            if not match:
                key_counter[('other',)][tag].append(value)
                

OSMParser(
    nodes_callback=process,
    ways_callback=process,
    relations_callback=process,
).parse('belarus-latest.osm.pbf')

In [9]:
data = []
for c in list(categories_rules) + ['other']:
    name_cnt = len(key_counter[(c,)]['name'])
    name_uniq = len(set(key_counter[(c,)]['name']))
    name_be_cnt = len(key_counter[(c,)]['name:be'])
    name_be_uniq = len(set(key_counter[(c,)]['name:be']))
    name_ru_cnt = len(key_counter[(c,)]['name:ru'])
    name_ru_uniq = len(set(key_counter[(c,)]['name:ru']))
    data.append([
        '#', c, 
        name_cnt, name_be_cnt, name_ru_cnt, name_be_cnt/(name_cnt or 1), name_ru_cnt/(name_cnt or 1),
        name_uniq, name_be_uniq, name_ru_uniq, name_be_uniq/(name_uniq or 1), name_ru_uniq/(name_uniq or 1),
    ])
    if c == 'other':
        continue
    for i, (k, eq, vv) in enumerate(categories_rules2[c]):
        if eq:
            tag = f'{k} = {list(vv)[0]}'
        else:
            tag = f'{k} = *'
        name_cnt = len(key_counter[(c, i)]['name'])
        name_uniq = len(set(key_counter[(c, i)]['name']))
        name_be_cnt = len(key_counter[(c, i)]['name:be'])
        name_be_uniq = len(set(key_counter[(c, i)]['name:be']))
        name_ru_cnt = len(key_counter[(c, i)]['name:ru'])
        name_ru_uniq = len(set(key_counter[(c, i)]['name:ru']))
        data.append([
            '', tag, 
            name_cnt, name_be_cnt, name_ru_cnt, name_be_cnt/(name_cnt or 1), name_ru_cnt/(name_cnt or 1),
            name_uniq, name_be_uniq, name_ru_uniq, name_be_uniq/(name_uniq or 1), name_ru_uniq/(name_uniq or 1),
        ])


df = pd.DataFrame(data, columns=[
    'lvl', 'category', 
    'all name', 'all name:be', 'all name:ru', 'all name:be%', 'all name:ru%',
    'uniq name', 'uniq name:be', 'uniq name:ru', 'uniq name:be%', 'uniq name:ru%',
])
(
    df
    .style
    .set_properties(subset=['category'], **{'text-align': 'left'})
    .background_gradient('YlOrRd', subset=[c for c in df.columns if c.endswith('%')], vmin=0, vmax=1)
    .format({f: '{:.3f}' for f in [c for c in df.columns if c.endswith('%')]})
    .apply(lambda row: [("font-weight: bold" if row.loc['lvl'] == '#' else '') for _ in row], axis=1)
)

,lvl,category,all name,all name:be,all name:ru,all name:be%,all name:ru%,uniq name,uniq name:be,uniq name:ru,uniq name:be%,uniq name:ru%
0,#,admin,2389,2260,2107,0.946,0.882,1571,1482,1487,0.943,0.947
1,,admin_level = 2,703,640,479,0.910,0.681,109,84,86,0.771,0.789
2,,admin_level = 4,73,73,73,1.000,1.000,40,39,39,0.975,0.975
3,,admin_level = 6,234,215,222,0.919,0.949,189,170,177,0.899,0.937
4,,admin_level = 8,1352,1306,1307,0.966,0.967,1234,1195,1192,0.968,0.966
5,,admin_level = 9,27,26,26,0.963,0.963,14,13,13,0.929,0.929
6,#,place,49941,49526,49404,0.992,0.989,17290,17069,16934,0.987,0.979
7,,place = city,31,31,31,1.000,1.000,16,16,16,1.000,1.000
8,,place = town,274,273,274,0.996,1.000,138,138,138,1.000,1.000
9,,place = village,5030,4990,5013,0.992,0.997,2229,2209,2214,0.991,0.993


## Падлічам статыстыку для выгрузкі ў postgis
- вынік будзе больш дакладным, але можа ня ўлічываць дачыненьні што не пераносяцца ў postgis

In [10]:
query_template = """
SELECT '{category}' AS category, {num} AS num, g.tags->'name' AS name, g.tags->'name:be' AS name_be, g.tags->'name:ru' AS name_ru
FROM {table} g
WHERE {condition}
"""
dependant_query_template = """
WITH tagged_table AS (
    SELECT field, COUNT(*) AS count
    FROM (
        SELECT DISTINCT osm_type, osm_id, field
        FROM (
            SELECT 'node' AS osm_type, g.osm_id, g.tags->'{field}' AS field 
            FROM planet_osm_point g
            INNER JOIN planet_osm_region p
            ON ST_Intersects(p.way, g.way)
            WHERE p.osm_id = -59065
            AND tags->'{field}' ~ '({cyr_regexp})'

            UNION ALL

            SELECT 'w+r' AS osm_type, g.osm_id, g.tags->'{field}' AS field
            FROM planet_osm_line g
            INNER JOIN planet_osm_region p
            ON ST_Intersects(p.way, g.way)
            WHERE p.osm_id = -59065
            AND tags->'{field}' ~ '({cyr_regexp})'
            
            UNION ALL

            SELECT 'w+r' AS osm_type, g.osm_id, g.tags->'{field}' AS field
            FROM planet_osm_roads g
            INNER JOIN planet_osm_region p
            ON ST_Intersects(p.way, g.way)
            WHERE p.osm_id = -59065
            AND tags->'{field}' ~ '({cyr_regexp})'

            UNION ALL

            SELECT 'w+r' AS osm_type, g.osm_id, g.tags->'{field}' AS field
            FROM planet_osm_polygon g
            INNER JOIN planet_osm_region p
            ON ST_Intersects(p.way, g.way)
            WHERE p.osm_id = -59065
            AND tags->'{field}' ~ '({cyr_regexp})'
        ) t1
    ) t2
    GROUP BY field
)
SELECT field, num, name, name_be, name_ru, SUM(count) AS count
FROM (
    SELECT '{field}' AS field, {num} AS num, c.count, 
        bool_or(p.name IS NOT NULL) AS name, 
        bool_or(p.name_be IS NOT NULL) AS name_be,
        bool_or(p.name_ru IS NOT NULL) AS name_ru
    FROM tagged_table c 
    LEFT JOIN {table} p 
    ON (c.field = p.name OR c.field = p.name_be OR c.field = p.name_ru)
    GROUP BY c.field, c.count
) t
GROUP BY field, num, name, name_be, name_ru
ORDER BY num
"""
tables = ['planet_osm_named_data']
cyr_regexp = '|'.join(cirylic_chars)

queries = []
exclude = defaultdict(lambda: [[], []])
for category, group in categories_rules2.items():
    conditions = []
    for i, (k, eq, vv) in enumerate(group):
        if vv:
            eq_str = 'IN' if eq else 'NOT IN'
            vv_str = ','.join(f"'{v}'" for v in vv)
            condition = f"g.tags->'{k}' {eq_str} ({vv_str})"
        elif not eq:
            condition = f"g.tags->'{k}' IS NOT NULL"
        else:
            raise ValueError()
        conditions.append(condition)
        exclude[k][eq].append(condition)
        for table in tables:
            query = query_template.format(category=category, num=i, table=table, condition=condition)
            queries.append(query)
    condition = ' OR '.join(f'({c})' for c in conditions)
    for table in tables:
        query = query_template.format(category=category, num=-1, table=table, condition=condition)
        queries.append(query)
condition = ' OR '.join(f"(g.tags->'{k}' IS NOT NULL)" for k, eq_c in exclude.items())
for table in tables:
    query = query_template.format(category='other', num=-1, table=table, condition=f'NOT ({condition})')
    queries.append(query)
for table in tables:
    query = query_template.format(category='TOTAL', num=-1, table=table, condition='TRUE')
    queries.append(query)
query = ' UNION ALL '.join(queries)

dependant_queries = []
for i, dependant in enumerate(dependants):
    for table in tables:
        query = dependant_query_template.format(num=i, table=table, field=dependant, cyr_regexp=cyr_regexp)
        dependant_queries.append(query)
dependant_query = ' UNION ALL '.join(dependant_queries)

print(len(queries), len(dependant_queries))


173 14


In [11]:
key_counter = defaultdict(lambda: defaultdict(list))
dependant_key_counter = defaultdict(Counter)

REGION_VIEW_SQL = """
CREATE MATERIALIZED VIEW IF NOT EXISTS planet_osm_region AS

SELECT
    osm_id,
    tags->'name' AS name,
    tags->'name:be' AS name_be,
    tags->'name:ru' AS name_ru,
    tags->'admin_level' AS admin_level,
    ST_Buffer(way, -0.000000001) AS way
FROM planet_osm_polygon
WHERE tags->'admin_level' IN ('2', '4', '6', '8', '9')
"""
REGION_WAY_INDEX_SQL = """
CREATE INDEX IF NOT EXISTS "planet_osm_region_way_idx" ON planet_osm_region USING GIST (way)
"""
REGION_ANALYZE_SQL = "ANALYZE planet_osm_region"

OSM_DATA_VIEW_SQL = f"""
CREATE MATERIALIZED VIEW IF NOT EXISTS planet_osm_named_data AS

SELECT
    g.osm_id AS osm_id,
    'node' AS osm_type,
    'point' AS kind,
    g.tags->'name' AS name,
    g.tags->'name:be' AS name_be,
    g.tags->'name:ru' AS name_ru,
    g.tags AS tags
FROM planet_osm_point g
INNER JOIN planet_osm_region p
ON ST_Intersects(p.way, g.way)
WHERE p.osm_id = -59065
AND g.tags->'name' ~ '({cyr_regexp})'

UNION ALL

SELECT
    ABS(g.osm_id) AS osm_id,
    CASE WHEN g.osm_id < 0 THEN 'relation' ELSE 'way' END AS osm_type,
    'line' AS kind,
    g.tags->'name' AS name,
    g.tags->'name:be' AS name_be,
    g.tags->'name:ru' AS name_ru,
    g.tags AS tags
FROM planet_osm_line g
INNER JOIN planet_osm_region p
ON ST_Intersects(p.way, g.way)
WHERE p.osm_id = -59065
AND g.tags->'name' ~ '({cyr_regexp})'

UNION ALL

SELECT
    ABS(g.osm_id) AS osm_id,
    CASE WHEN g.osm_id < 0 THEN 'relation' ELSE 'way' END AS osm_type,
    'road' AS kind,
    g.tags->'name' AS name,
    g.tags->'name:be' AS name_be,
    g.tags->'name:ru' AS name_ru,
    g.tags AS tags
FROM planet_osm_roads g
INNER JOIN planet_osm_region p
ON ST_Intersects(p.way, g.way)
WHERE p.osm_id = -59065
AND g.tags->'name' ~ '({cyr_regexp})'

UNION ALL

SELECT
    ABS(g.osm_id) AS osm_id,
    CASE WHEN g.osm_id < 0 THEN 'relation' ELSE 'way' END AS osm_type,
    'poly' AS kind,
    g.tags->'name' AS name,
    g.tags->'name:be' AS name_be,
    g.tags->'name:ru' AS name_ru,
    g.tags AS tags
FROM planet_osm_polygon g
INNER JOIN planet_osm_region p
ON ST_Intersects(p.way, g.way)
WHERE p.osm_id = -59065
AND g.tags->'name' ~ '({cyr_regexp})'
"""
OSM_DATA_OSM_ID_TYPE_INDEX_SQL = """
CREATE INDEX IF NOT EXISTS "planet_osm_named_data_osm_id_type_idx" ON planet_osm_named_data (osm_id, osm_type)
"""
OSM_DATA_TAGS_INDEX_SQL = """
CREATE INDEX IF NOT EXISTS "planet_osm_named_data_tags_idx" ON planet_osm_named_data USING GIN (tags)
"""
OSM_DATA_NAME_INDEX_SQL = """
CREATE INDEX IF NOT EXISTS "planet_osm_named_data_name_idx" ON planet_osm_named_data (name)
"""
OSM_DATA_NAME_BE_INDEX_SQL = """
CREATE INDEX IF NOT EXISTS "planet_osm_named_data_name_be_idx" ON planet_osm_named_data (name_be)
"""
OSM_DATA_NAME_RU_INDEX_SQL = """
CREATE INDEX IF NOT EXISTS "planet_osm_named_data_name_ru_idx" ON planet_osm_named_data (name_ru)
"""
OSM_DATA_ANALYZE_SQL = "ANALYZE planet_osm_named_data"


postgres_params = {
    'host': os.environ['POSTGRES_HOST'],
    'dbname': os.environ['POSTGRES_DB'],
    'user': os.environ['POSTGRES_USER'],
    'password': os.environ['POSTGRES_PASSWORD'],
}

try:
    from belarus_utils import PostgisSearchReadEngine, OverpassApiSearchEnigne
    postgis_api = PostgisSearchReadEngine(**postgres_params)
    overpass_api = OverpassApiSearchEnigne(cache=True)
    
    print('type = street')
    postgis_api.insert_extra_relations(overpass_api.search({'type': ['street']}))
    
    print('type = associatedStreet')
    postgis_api.insert_extra_relations(overpass_api.search({'type': ['associatedStreet']}))
    
    print('type = route')
    postgis_api.insert_extra_relations(overpass_api.search({'type': ['route']}))
    
    print('type = route_master')
    postgis_api.insert_extra_relations(overpass_api.search({'type': ['route_master']}))
    
    print('type = waterway')
    postgis_api.insert_extra_relations(overpass_api.search({'type': ['waterway']}))
except Exception as err:
    print(err)

with psycopg2.connect(**postgres_params) as conn:
    conn.autocommit = True
    with conn.cursor() as cur:
        cur.execute(REGION_VIEW_SQL)
        cur.execute(REGION_WAY_INDEX_SQL)
        cur.execute(REGION_ANALYZE_SQL)

        cur.execute(OSM_DATA_VIEW_SQL)
        cur.execute(OSM_DATA_OSM_ID_TYPE_INDEX_SQL)
        cur.execute(OSM_DATA_TAGS_INDEX_SQL)
        cur.execute(OSM_DATA_NAME_INDEX_SQL)
        cur.execute(OSM_DATA_NAME_BE_INDEX_SQL)
        cur.execute(OSM_DATA_NAME_RU_INDEX_SQL)
        cur.execute(OSM_DATA_ANALYZE_SQL)

with psycopg2.connect(**postgres_params) as conn:
    with conn.cursor() as cur:
        for i, query in enumerate(queries, 1):
            cur.execute(query)
            records = cur.fetchall()
            for category, num, name, name_be, name_ru in records:
                key = (category,) if num == -1 else (category, num)
                key_counter[key]['name'].append(name)
                if name_be is not None:
                    key_counter[key]['name:be'].append(name_be)
                if name_ru is not None:
                    key_counter[key]['name:ru'].append(name_ru)
                    
                if name == name_be == name_ru:
                    key_counter[key]['res_both'].append(name)
                elif name == name_be and name_ru is not None:
                    key_counter[key]['res_be_ru'].append(name)
                elif name == name_be:
                    key_counter[key]['res_be'].append(name)
                elif name == name_ru and name_be is not None:
                    key_counter[key]['res_ru_be'].append(name)
                elif name == name_ru:
                    key_counter[key]['res_ru'].append(name)
                elif name_be is not None and name_ru is not None:
                    key_counter[key]['res_other_both'].append(name)
                elif name_be is not None:
                    key_counter[key]['res_other_be'].append(name)
                elif name_ru is not None:
                    key_counter[key]['res_other_ru'].append(name)
                else:
                    key_counter[key]['res_none'].append(name)
        for i, query in enumerate(dependant_queries, 1):
            cur.execute(query)
            records = cur.fetchall()
            for field, num, name, name_be, name_ru, count in records:
                if name and name_be and name_ru:
                    dependant_key_counter[field]['all'] += count
                elif name and name_be and not name_ru:
                    dependant_key_counter[field]['be'] += count
                elif name and not name_be and name_ru:
                    dependant_key_counter[field]['ru'] += count
                elif name:
                    dependant_key_counter[field]['name'] += count
                else:
                    dependant_key_counter[field]['not found'] += count


type = street
type = associatedStreet
type = route
type = route_master
type = waterway


In [12]:
data = []
for c in list(categories_rules) + ['other', 'TOTAL']:
    name_cnt = len(key_counter[(c,)]['name'])
    name_uniq = len(set(key_counter[(c,)]['name']))
    name_be_cnt = len(key_counter[(c,)]['name:be'])
    name_be_uniq = len(set(key_counter[(c,)]['name:be']))
    name_ru_cnt = len(key_counter[(c,)]['name:ru'])
    name_ru_uniq = len(set(key_counter[(c,)]['name:ru']))
    data.append([
        '#', c, 
        name_cnt, name_be_cnt, name_ru_cnt, name_be_cnt/(name_cnt or 1), name_ru_cnt/(name_cnt or 1),
        name_uniq, name_be_uniq, name_ru_uniq, name_be_uniq/(name_uniq or 1), name_ru_uniq/(name_uniq or 1),
    ])
    if c in {'other', 'TOTAL'}:
        continue
    for i, (k, eq, vv) in enumerate(categories_rules2[c]):
        if eq:
            tag = f'{k} = {list(vv)[0]}'
        else:
            tag = f'{k} = *'
        name_cnt = len(key_counter[(c, i)]['name'])
        name_uniq = len(set(key_counter[(c, i)]['name']))
        name_be_cnt = len(key_counter[(c, i)]['name:be'])
        name_be_uniq = len(set(key_counter[(c, i)]['name:be']))
        name_ru_cnt = len(key_counter[(c, i)]['name:ru'])
        name_ru_uniq = len(set(key_counter[(c, i)]['name:ru']))
        data.append([
            '', tag, 
            name_cnt, name_be_cnt, name_ru_cnt, name_be_cnt/(name_cnt or 1), name_ru_cnt/(name_cnt or 1),
            name_uniq, name_be_uniq, name_ru_uniq, name_be_uniq/(name_uniq or 1), name_ru_uniq/(name_uniq or 1),
        ])

df = pd.DataFrame(data, columns=[
    'lvl', 'category', 
    'all name', 'all name:be', 'all name:ru', 'all name:be%', 'all name:ru%',
    'uniq name', 'uniq name:be', 'uniq name:ru', 'uniq name:be%', 'uniq name:ru%',
])
(
    df
    .style
    .set_properties(subset=['category'], **{'text-align': 'left'})
    .background_gradient('YlOrRd', subset=[c for c in df.columns if c.endswith('%')], vmin=0, vmax=1)
    .format({f: '{:.3f}' for f in [c for c in df.columns if c.endswith('%')]})
    .apply(lambda row: [("font-weight: bold" if row.loc['lvl'] == '#' else '') for _ in row], axis=1)
)

,lvl,category,all name,all name:be,all name:ru,all name:be%,all name:ru%,uniq name,uniq name:be,uniq name:ru,uniq name:be%,uniq name:ru%
0,#,admin,6874,6874,6874,1.000,1.000,1393,1397,1392,1.003,0.999
1,,admin_level = 2,4,4,4,1.000,1.000,3,3,3,1.000,1.000
2,,admin_level = 4,323,323,323,1.000,1.000,32,31,31,0.969,0.969
3,,admin_level = 6,1304,1304,1304,1.000,1.000,165,165,165,1.000,1.000
4,,admin_level = 8,5139,5139,5139,1.000,1.000,1191,1195,1191,1.003,1.000
5,,admin_level = 9,104,104,104,1.000,1.000,13,13,13,1.000,1.000
6,#,place,114282,114275,114277,1.000,1.000,16946,16971,16805,1.001,0.992
7,,place = city,119,119,119,1.000,1.000,16,16,16,1.000,1.000
8,,place = town,677,677,677,1.000,1.000,137,138,137,1.007,1.000
9,,place = village,12280,12277,12277,1.000,1.000,2198,2207,2195,1.004,0.999


In [13]:
data = []
for c in list(categories_rules) + ['other', 'TOTAL']:
    name_both_cnt = len(key_counter[(c,)]['res_both'])
    name_be_ru_cnt = len(key_counter[(c,)]['res_be_ru'])
    name_be_cnt = len(key_counter[(c,)]['res_be'])
    name_ru_be_cnt = len(key_counter[(c,)]['res_ru_be'])
    name_ru_cnt = len(key_counter[(c,)]['res_ru'])
    name_other_both_cnt = len(key_counter[(c,)]['res_other_both'])
    name_other_be_cnt = len(key_counter[(c,)]['res_other_be'])
    name_other_ru_cnt = len(key_counter[(c,)]['res_other_ru'])
    name_none_cnt = len(key_counter[(c,)]['res_none'])
    total = (
        name_both_cnt + name_be_ru_cnt + name_be_cnt + name_ru_be_cnt + name_ru_cnt + 
        name_other_both_cnt + name_other_be_cnt + name_other_ru_cnt + 
        name_none_cnt
    )
    data.append([
        '#', c, 
        
        name_both_cnt, 
        name_be_ru_cnt, name_be_cnt, 
        name_ru_be_cnt, name_ru_cnt, 
        name_other_both_cnt, name_other_be_cnt, name_other_ru_cnt, 
        name_none_cnt, 
        
        name_both_cnt/(total or 1), 
        name_be_ru_cnt/(total or 1), name_be_cnt/(total or 1), 
        name_ru_be_cnt/(total or 1), name_ru_cnt/(total or 1), 
        name_other_both_cnt/(total or 1), name_other_be_cnt/(total or 1), name_other_ru_cnt/(total or 1), 
        name_none_cnt/(total or 1),
    ])
    if c in {'other', 'TOTAL'}:
        continue
    for i, (k, eq, vv) in enumerate(categories_rules2[c]):
        if eq:
            tag = f'{k} = {list(vv)[0]}'
        else:
            tag = f'{k} = *'
            
        name_both_cnt = len(key_counter[(c, i)]['res_both'])
        name_be_ru_cnt = len(key_counter[(c, i)]['res_be_ru'])
        name_be_cnt = len(key_counter[(c, i)]['res_be'])
        name_ru_be_cnt = len(key_counter[(c, i)]['res_ru_be'])
        name_ru_cnt = len(key_counter[(c, i)]['res_ru'])
        name_other_both_cnt = len(key_counter[(c, i)]['res_other_both'])
        name_other_be_cnt = len(key_counter[(c, i)]['res_other_be'])
        name_other_ru_cnt = len(key_counter[(c, i)]['res_other_ru'])
        name_none_cnt = len(key_counter[(c, i)]['res_none'])
        total = (
            name_both_cnt + name_be_ru_cnt + name_be_cnt + name_ru_be_cnt + name_ru_cnt + 
            name_other_both_cnt + name_other_be_cnt + name_other_ru_cnt + 
            name_none_cnt
        )
        data.append([
            '', tag,
            
            name_both_cnt, 
            name_be_ru_cnt, name_be_cnt, 
            name_ru_be_cnt, name_ru_cnt, 
            name_other_both_cnt, name_other_be_cnt, name_other_ru_cnt, 
            name_none_cnt, 

            name_both_cnt/(total or 1), 
            name_be_ru_cnt/(total or 1), name_be_cnt/(total or 1), 
            name_ru_be_cnt/(total or 1), name_ru_cnt/(total or 1), 
            name_other_both_cnt/(total or 1), name_other_be_cnt/(total or 1), name_other_ru_cnt/(total or 1), 
            name_none_cnt/(total or 1),
        ])


df = pd.DataFrame(data, columns=[
    'lvl', 'category', 
    'name be=ru', 'name be+ru', 'name be', 'name ru+be', 'name ru', 
    'other both', 'other be', 'other ru', 'no lang',
    'name be=ru%', 'name be+ru%', 'name be%', 'name ru+be%', 'name ru%', 
    'other both%', 'other be%', 'other ru%', 'no lang%',
])
(
    df
    .style
    .set_properties(subset=['category'], **{'text-align': 'left'})
    .set_properties(subset=['name be=ru', 'name be+ru', 'name be'], **{'background-color': '#d9ead3'})
    .set_properties(subset=['name ru+be'], **{'background-color': '#fff2cc'})
    .set_properties(subset=['name ru', 'other both', 'other be', 'other ru', 'no lang'], **{'background-color': '#f4cccc'})
    .background_gradient('YlOrRd', subset=[c for c in df.columns if c.endswith('%')], vmin=0, vmax=1)
    .format({f: '{:.3f}' for f in [c for c in df.columns if c.endswith('%')]})
    .apply(lambda row: [("font-weight: bold" if row.loc['lvl'] == '#' else '') for _ in row], axis=1)
)

,lvl,category,name be=ru,name be+ru,name be,name ru+be,name ru,other both,other be,other ru,no lang,name be=ru%,name be+ru%,name be%,name ru+be%,name ru%,other both%,other be%,other ru%,no lang%
0,#,admin,146,4,0,6724,0,0,0,0,0,0.021,0.001,0.000,0.978,0.000,0.000,0.000,0.000,0.000
1,,admin_level = 2,1,2,0,1,0,0,0,0,0,0.250,0.500,0.000,0.250,0.000,0.000,0.000,0.000,0.000
2,,admin_level = 4,34,2,0,287,0,0,0,0,0,0.105,0.006,0.000,0.889,0.000,0.000,0.000,0.000,0.000
3,,admin_level = 6,51,0,0,1253,0,0,0,0,0,0.039,0.000,0.000,0.961,0.000,0.000,0.000,0.000,0.000
4,,admin_level = 8,60,0,0,5079,0,0,0,0,0,0.012,0.000,0.000,0.988,0.000,0.000,0.000,0.000,0.000
5,,admin_level = 9,0,0,0,104,0,0,0,0,0,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000
6,#,place,9364,198,0,104713,2,0,0,0,5,0.082,0.002,0.000,0.916,0.000,0.000,0.000,0.000,0.000
7,,place = city,22,0,0,97,0,0,0,0,0,0.185,0.000,0.000,0.815,0.000,0.000,0.000,0.000,0.000
8,,place = town,56,0,0,621,0,0,0,0,0,0.083,0.000,0.000,0.917,0.000,0.000,0.000,0.000,0.000
9,,place = village,1186,0,0,11091,0,0,0,0,3,0.097,0.000,0.000,0.903,0.000,0.000,0.000,0.000,0.000


In [14]:
data = []
for dependant in list(dependants):
    name_all_cnt = dependant_key_counter[dependant]['all']
    name_be_cnt = dependant_key_counter[dependant]['be']
    name_ru_cnt = dependant_key_counter[dependant]['ru']
    name_name_cnt = dependant_key_counter[dependant]['name']
    name_no_cnt = dependant_key_counter[dependant]['not found']
    total = (name_all_cnt + name_be_cnt + name_ru_cnt + name_name_cnt + name_no_cnt)
    data.append([
        '#', dependant, 
        
        name_all_cnt, 
        name_be_cnt, 
        name_ru_cnt, 
        name_name_cnt, 
        name_no_cnt, 
        
        name_all_cnt/(total or 1), 
        name_be_cnt/(total or 1),
        name_ru_cnt/(total or 1),
        name_name_cnt/(total or 1),
        name_no_cnt/(total or 1),
    ])


df = pd.DataFrame(data, columns=[
    'lvl', 'category', 
    'name all', 'name be', 'name ru', 'name only', 'not found', 
    'name all%', 'name be%', 'name ru%', 'name only%', 'not found%', 
])
(
    df
    .style
    .set_properties(subset=['category'], **{'text-align': 'left'})
    .set_properties(subset=['name all', 'name be'], **{'background-color': '#d9ead3'})
    .set_properties(subset=['name ru', 'name only'], **{'background-color': '#fff2cc'})
    .set_properties(subset=['not found'], **{'background-color': '#f4cccc'})
    .background_gradient('YlOrRd', subset=[c for c in df.columns if c.endswith('%')], vmin=0, vmax=1)
    .format({f: '{:.3f}' for f in [c for c in df.columns if c.endswith('%')]})
    .apply(lambda row: [("font-weight: bold" if row.loc['lvl'] == '#' else '') for _ in row], axis=1)
)

,lvl,category,name all,name be,name ru,name only,not found,name all%,name be%,name ru%,name only%,not found%
0,#,addr:region,54988,0,0,0,0,1.000,0.000,0.000,0.000,0.000
1,#,addr:district,54682,0,0,0,108,0.998,0.000,0.000,0.000,0.002
2,#,addr:subdistrict,12994,0,0,0,0,1.000,0.000,0.000,0.000,0.000
3,#,addr:city,114361,0,130,0,2,0.999,0.000,0.001,0.000,0.000
4,#,addr:place,36954,0,2,0,54,0.998,0.000,0.000,0.000,0.001
5,#,addr:street,860120,11,1507,81,354,0.998,0.000,0.002,0.000,0.000
6,#,addr2:street,629,0,0,0,0,1.000,0.000,0.000,0.000,0.000
7,#,from,724,0,15,2,271,0.715,0.000,0.015,0.002,0.268
8,#,to,705,0,12,3,189,0.776,0.000,0.013,0.003,0.208
9,#,via,68,0,0,0,40,0.630,0.000,0.000,0.000,0.370
